# Read all images

In [22]:
import os
import random
import shutil

all_data = {'crazing': [], 'inclusion': [], 'patches': [], 'pitted': [], 'rolled-in': [], 'scratches': []}

for f in os.listdir('origin/IMAGES'):
    all_data[f.split('_')[0]].append(f)

# randomly select validation data
VALID_NUM = 60
valid_set = {'crazing': [], 'inclusion': [], 'patches': [], 'pitted': [], 'rolled-in': [], 'scratches': []}
for c in all_data:
    for i in range(VALID_NUM):
        selected = random.choice(all_data[c])
        valid_set[c].append(selected)
        all_data[c].remove(selected)
        

# randomly select evaluation data
EVALUATE_NUM = 60
eval_set = {'crazing': [], 'inclusion': [], 'patches': [], 'pitted': [], 'rolled-in': [], 'scratches': []}
for c in all_data:
    for i in range(EVALUATE_NUM):
        selected = random.choice(all_data[c])
        eval_set[c].append(selected)
        all_data[c].remove(selected)

# clean the dirs
shutil.rmtree('test')
shutil.rmtree('train')
shutil.rmtree('valid')
os.mkdir('train')
os.mkdir('test')
os.mkdir('valid')

# copy image to the dir according to the list
for c in all_data:
    for f in all_data[c]:
        shutil.copy(f'origin/IMAGES/{f}', 'train')
    for f in valid_set[c]:
        shutil.copy(f'origin/IMAGES/{f}', 'valid')
    for f in eval_set[c]:
        shutil.copy(f'origin/IMAGES/{f}', 'test')

# Convert annotation to YOLO format

In [38]:
import xml.etree.ElementTree as ET
import os

classes = ['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']

def convert_bndbox(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert(in_name, out_name):
    
    in_file = open(in_name, 'r')
    out_file = open(out_name, 'w') # .txt
    
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')  
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls not in classes :
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')   
        box = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        converted_box = convert_bndbox((w,h), box)
        
        out_file.write(str(cls_id) + " " + " ".join([str(s) for s in converted_box]) + '\n')

for folder in ['train', 'valid', 'test']:
    for f in os.listdir(f'{folder}/images'):
        file_name = f.split('.jpg')[0]
        convert(f'origin/ANNOTATIONS/{file_name}.xml', f'{folder}/labels/{file_name}.txt')


# Make image list

In [30]:
for folder in ['train', 'valid', 'test']:
    out_file = open(f'/home/cysun/Projects/dl_system/DL_system/DLSR-final/PyTorch-YOLOv3/data/custom/{folder}.txt', 'w')
    for f in os.listdir(f'{folder}/images'):
        out_file.write(f"../../../train_data/NEU-DET/{folder}/images/{f}\n")
    out_file.close()

## k-means clustering for anchor boxes 
* Reference: https://lars76.github.io/object-detection/k-means-anchor-boxes/

In [53]:
import numpy as np

def iou(box, clusters):
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou_ = intersection / (box_area + cluster_area - intersection)

    return iou_

def kmeans(boxes, k, dist=np.median):
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    clusters = boxes[np.random.choice(rows, k, replace=False)]

    while True:
        for row in range(rows):
            distances[row] = 1 - iou(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

def get_annoation_boxes(in_name):
    in_file = open(in_name, 'r')
    tree = ET.parse(in_file)
    root = tree.getroot()
    boxes = []
    
    for obj in root.iter('object'):
        xmlbox = obj.find('bndbox')   
#         boxes.append((float(xmlbox.find('xmin').text), float(xmlbox.find('ymin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymax').text)))
        boxes.append((float(xmlbox.find('xmax').text) - float(xmlbox.find('xmin').text), float(xmlbox.find('ymax').text) - float(xmlbox.find('ymin').text)))
    
    return boxes

boxes = []

# get boxes from annoation files
for f in os.listdir(f'origin/ANNOTATIONS'):
    boxes = boxes + get_annoation_boxes(f"origin/ANNOTATIONS/{f}")

# compute k-means
print(kmeans(np.array(boxes), 9))    

[[ 46.  39.]
 [ 28. 185.]
 [172.  28.]
 [154. 193.]
 [ 74.  63.]
 [ 68. 133.]
 [134.  75.]
 [ 37.  77.]
 [ 19.  42.]]
